In [2]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta, datetime
import time
import plotly.express as px

# Lấy data từ API của WeatherBit cho chất lượng không khí
thông tin được lấy ở đây là thông số không khí từ ngày 1/1/2023  đến ngày 1/12/2025 tại Cần Thơ

In [ ]:
API_KEY = "7db8deefb8794f2eae88d63d31f90eee"
BASE_URL_AIRQUALITY = "https://api.weatherbit.io/v2.0/history/airquality"
LAT = 10.047264
LON = 105.768115
START_DATE = datetime(2023, 1, 1)
END_DATE = datetime(2025, 12, 1)
FORMAT_DATE = "%Y-%m-%d"
all_data = []


Fetching data from 2023-01-01 to 2023-01-21
No data found for the period 2023-01-01 to 2023-01-21


In [ ]:
current = START_DATE
while current < END_DATE:
    block_end = current + timedelta(days=29)
    if block_end > END_DATE:
        block_end = END_DATE

    params = {
        "lat": LAT, 
        "lon": LON,
        "start_date": current.strftime(FORMAT_DATE),
        "end_date": block_end.strftime(FORMAT_DATE),
        "key": API_KEY
    }
    print(f"Fetching data from {params['start_date']} to {params['end_date']}")
    response = requests.get(BASE_URL_AIRQUALITY, params=params)

    if response.status_code == 200:
        json_data = response.json()
        if json_data.get("data"):
            all_data.extend(json_data["data"])
            print(f"Retrieved {len(json_data['data'])} records.")
    else:
        print(f"No data found for the period {params['start_date']} to {params['end_date']}")
        break
    
    current = block_end + timedelta(days=1)
    time.sleep(7)

In [4]:
df = pd.DataFrame(all_data)
df.drop(columns=["datetime"], inplace=True) 
# xử lý time series
df["timestamp_local"] = pd.to_datetime(df["timestamp_local"])
df = df.sort_values("timestamp_local").drop_duplicates()
df.set_index("timestamp_local", inplace=True)

KeyError: "['datetime'] not found in axis"

In [29]:
df.to_csv("cantho_air_quality_2023_2025.csv", index=True)

In [20]:
df.head()

,aqi,co,no2,o3,pm10,pm25,so2,timestamp_utc,ts
timestamp_local,,,,,,,,,
2023-01-01 00:00:00,152,432.8,57.3,22.3,83.3,56.67,37.7,2022-12-31T17:00:00,1672506000
2023-01-01 01:00:00,150,402.1,54.0,20.0,79.0,54.00,32.0,2022-12-31T18:00:00,1672509600
2023-01-01 02:00:00,134,361.2,45.3,24.3,70.3,48.00,31.7,2022-12-31T19:00:00,1672513200
2023-01-01 03:00:00,118,320.4,36.7,28.7,61.7,42.00,31.3,2022-12-31T20:00:00,1672516800
2023-01-01 04:00:00,102,279.5,28.0,33.0,53.0,36.00,31.0,2022-12-31T21:00:00,1672520400


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 20661 entries, 2023-01-01 00:00:00 to 2025-11-30 00:00:00
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   aqi            20661 non-null  int64  
 1   co             20661 non-null  float64
 2   no2            20661 non-null  float64
 3   o3             20661 non-null  float64
 4   pm10           20661 non-null  float64
 5   pm25           20661 non-null  float64
 6   so2            20661 non-null  float64
 7   timestamp_utc  20661 non-null  object 
 8   ts             20661 non-null  int64  
dtypes: float64(6), int64(2), object(1)
memory usage: 1.6+ MB


# Lấy dữ liệu từ thời tiết

In [ ]:
API_KEY = "970fcc7ba75842cabe211d3f49feecd4"
BASE_URL_HOURLY = "https://api.weatherbit.io/v2.0/history/hourly"
LAT = 10.047264
LON = 105.768115
START_DATE = datetime(2023, 1, 1)
END_DATE = datetime(2025, 12, 1)
FORMAT_DATE = "%Y-%m-%d"
all_data_weather_hourly = []


In [ ]:

current = START_DATE
while current < END_DATE:
    block_end = current + timedelta(days=29)
    if block_end > END_DATE:
        block_end = END_DATE
    params = {
        "lat": LAT, 
        "lon": LON,
        "start_date": current.strftime("%Y-%m-%d"),
        "end_date": block_end.strftime("%Y-%m-%d"),
        "tz":"local",
        "units":"M",
        "key": API_KEY
    }
    print(f"Fetching hourly data from {params['start_date']} to {params['end_date']}")
    response = requests.get(BASE_URL_HOURLY, params=params)
    if response.status_code == 200:
        json_data = response.json()
        if json_data.get("data"):
            all_data_weather_hourly.extend(json_data["data"])
        else:
            print(f"No hourly data found for the period {params['start_date']} to {params['end_date']}")
    else:
        print(f"Failed to fetch data: {response.status_code}")
        break

    current = block_end + timedelta(days=1)
    time.sleep(7)